In [1]:
from openff.toolkit import ForceField, Molecule
from openmm import openmm
from openmm import unit
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Draw
import numpy as np

In [2]:
class Reporter(openmm.MinimizationReporter):
    interval = 10 # report interval
    energies = [] # array to record progress
    def report(self, iteration, x, grad, args):
        # print current system energy to screen 
        if iteration % self.interval == 0:
            print(iteration, args['system energy'])

        # save energy at each iteration to an array we can use later
        self.energies.append(args['system energy'])

        # The report method must return a bool specifying if minimization should be stopped. 
        return False

In [3]:
mol = Molecule.from_smiles('CCCC(O)CC(N)CCC', allow_undefined_stereo=True)
mol.generate_conformers()
# mol.assign_partial_charges(partial_charge_method='am1bccelf10')

In [4]:
print(f"Number of atoms in molecule: {mol.n_atoms}")
print(f"Number of conformers: {len(mol.conformers)}")


Number of atoms in molecule: 32
Number of conformers: 1


In [5]:
forcefield = ForceField('lee-krimm-force-field.offxml', load_plugins = True, allow_cosmetic_attributes = True)
# forcefield = ForceField('openff-2.2.0.offxml')

In [6]:
system = forcefield.create_openmm_system(mol.to_topology())
# use interchange for system creation

ValidationError: 1 validation error for Potential
parameters -> s
  Value 1.0 needs to be tagged with a unit (type=value_error.missingunit)

In [ ]:
for i in range(system.getNumForces()):
    force = system.getForce(i)
    print(f"Force {i}: {type(force)}")


In [ ]:
print(system.getNumParticles()) 

In [ ]:
integrator = openmm.LangevinIntegrator(
    300.0 * unit.kelvin, 1.0 / unit.picosecond, 2.0 * unit.femtosecond
)
platform = openmm.Platform.getPlatformByName("Reference")
context = openmm.Context(system, integrator, platform)

initial_positions = mol.conformers[0].to_openmm()
context.setPositions(initial_positions)

# Create an instance of our reporter
reporter = Reporter()

# Perform local energy minimization
print("Minimizing energy...")
openmm.LocalEnergyMinimizer.minimize(
    context, tolerance=1e-2, maxIterations=1500, reporter=reporter
)

In [ ]:
plt.plot(reporter.energies)
plt.xlabel("Iteration")
plt.ylabel("Energy (kJ/mol)")
plt.title("Energy Minimization Progress")
plt.show()

In [ ]:
xml_file_name = "lee_krimm_benchmark_system.xml"
with open(xml_file_name, "w") as xml_file:
    xml_file.write(openmm.XmlSerializer.serialize(system))